In [1]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F 
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import time
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
device

device(type='cuda')

In [3]:
def save_checkpoint(epoch, model, optimizer):
    """
    Save model checkpoint.
    :param epoch: epoch number
    :param model: model
    :param optimizer: optimizer
    """
    state = {'epoch': epoch,
             'model': model,
             'optimizer': optimizer}
    filename = 'checkpoint_resnet18_cutout.pth.tar'
    torch.save(state, filename)

class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [4]:
# 数据集读取

epochs = 200
pre_epoch = 0
BATCH_SIZE = 128
LR = 0.01

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = torchvision.datasets.CIFAR10(root='data', train=True, download=False, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='data', train=False, download=False, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)    #核为4，步长为4
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [6]:
class Cutout(object):
    """Randomly mask out one or more patches from an image.
    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img

In [7]:
checkpoint = 'checkpoint_resnet18_cutout.pth.tar'
print_freq = 20

def main():
    """
    Training.
    """
    
    global start_epoch, classes, epoch, checkpoint
    
    # 初始化模型
    
    if checkpoint is None:
        start_epoch = 0
        model = ResNet18()
        optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    else:
        checkpoint = torch.load(checkpoint, map_location = 'cuda')
        start_epoch = checkpoint['epoch'] + 1
        print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(start_epoch, epochs):
        
        if epoch == 100:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.1, momentum=0.9, weight_decay=5e-4)
        if epoch == 150:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.01, momentum=0.9, weight_decay=5e-4)
        
        train(train_loader = train_loader,
             model = model,
             criterion=criterion,
             optimizer=optimizer,
             epoch=epoch)
        save_checkpoint(epoch, model, optimizer)
        evaluate(test_loader, model)
        

def train(train_loader, model, criterion, optimizer, epoch):
    
    model = model.train()
    
    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()
    total = 0
    correct = 0
    # 作用cutout
    cut = Cutout(n_holes = 1, length = 16)
    for i, data in enumerate(train_loader, 0):
        length = len(train_loader)
        inputs, labels = data
        for dd in range(inputs.size(0)):
            if random.random()<0.5:
                inputs[dd] = cut(inputs[dd])
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # forward + backward
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.update(loss.item(), inputs.size(0))
        batch_time.update(time.time() - start)
        start = time.time()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1 = open("train_resnet18_cutout.txt", "a")
            f1.write('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t\n'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1.close()
            print(correct/total)
            f2 = open("train_acc_resnet18_cutout.txt", "a")
            f2.write('测试分类准确率为：%.3f%%\n' % (100 * correct / total))
            f2.close()
def evaluate(test_loader, model):
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # 取得分最高的那个类 (outputs.data的索引号)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        print('测试分类准确率为：%.3f%%' % (100 * correct / total))
        f3 = open("test_acc_resnet18_cutout.txt", "a")
        f3.write('测试分类准确率为：%.3f%%\n' % (100 * correct / total))
        f3.close()
        acc = 100. * correct / total
        best_acc = 85
        if acc > best_acc:
            f3 = open("best_acc_resnet18_cutout.txt", "w")
            f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
            f3.close()
            best_acc = acc
if __name__ == '__main__':
    main()


Loaded checkpoint from epoch 200.



In [7]:
device

device(type='cpu')

In [10]:
for i, data in enumerate(train_loader, 0):
    inputs, labels = data
    print(inputs)

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x0000017A41FB3940>>
Traceback (most recent call last):
  File "e:\miniconda\miniconda\lib\site-packages\torch\utils\data\dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "e:\miniconda\miniconda\lib\site-packages\torch\utils\data\dataloader.py", line 1291, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


tensor([[[[ 1.0602,  1.0408,  1.0214,  ..., -2.4291, -2.4291, -2.4291],
          [ 1.0214,  1.0021,  0.9827,  ..., -2.4291, -2.4291, -2.4291],
          [ 0.9633,  0.9439,  0.9439,  ..., -2.4291, -2.4291, -2.4291],
          ...,
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291]],

         [[ 1.1218,  1.1021,  1.1021,  ..., -2.4183, -2.4183, -2.4183],
          [ 1.1021,  1.0824,  1.0824,  ..., -2.4183, -2.4183, -2.4183],
          [ 1.0628,  1.0431,  1.0234,  ..., -2.4183, -2.4183, -2.4183],
          ...,
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183]],

         [[ 1.2905,  1.2709,  1.2514,  ..., -2.2214, -2.2214, -2.2214],
          [ 1.2709,  1.2514,  

          [-2.2214, -2.2214, -2.2214,  ..., -2.2214, -2.2214, -2.2214]]]])
tensor([[[[-0.3936, -0.5100, -0.6069,  ..., -2.4291, -2.4291, -2.4291],
          [-0.8589, -0.8977, -0.9170,  ..., -2.4291, -2.4291, -2.4291],
          [-0.9364, -0.8783, -0.9558,  ..., -2.4291, -2.4291, -2.4291],
          ...,
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291]],

         [[-0.3532, -0.4516, -0.5302,  ..., -2.4183, -2.4183, -2.4183],
          [-0.8252, -0.8449, -0.8056,  ..., -2.4183, -2.4183, -2.4183],
          [-0.8449, -0.7662, -0.8449,  ..., -2.4183, -2.4183, -2.4183],
          ...,
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183]],

         [[-0.2118, -0.2704

          [-2.2214, -2.2214, -2.2214,  ..., -2.2214, -2.2214, -2.2214]]]])
tensor([[[[-2.4291e+00, -2.4291e+00, -2.4291e+00,  ..., -2.4291e+00,
           -2.4291e+00, -2.4291e+00],
          [-2.4291e+00, -2.4291e+00, -2.4291e+00,  ..., -2.4291e+00,
           -2.4291e+00, -2.4291e+00],
          [-2.4291e+00, -2.4291e+00, -2.4291e+00,  ..., -2.4291e+00,
           -2.4291e+00, -2.4291e+00],
          ...,
          [-2.4291e+00, -2.4291e+00, -3.5488e-01,  ...,  9.0974e-02,
            4.0113e-01,  4.5929e-01],
          [-2.4291e+00, -2.4291e+00, -2.9672e-01,  ..., -1.9980e-01,
            1.2974e-01,  1.4913e-01],
          [-2.4291e+00, -2.4291e+00, -2.5796e-01,  ..., -8.0073e-01,
           -2.1919e-01,  1.2974e-01]],

         [[-2.4183e+00, -2.4183e+00, -2.4183e+00,  ..., -2.4183e+00,
           -2.4183e+00, -2.4183e+00],
          [-2.4183e+00, -2.4183e+00, -2.4183e+00,  ..., -2.4183e+00,
           -2.4183e+00, -2.4183e+00],
          [-2.4183e+00, -2.4183e+00, -2.4183e+00,  .

           -2.2214e+00, -2.2214e+00]]]])
tensor([[[[-2.4291e+00, -2.4291e+00, -2.4291e+00,  ..., -2.4291e+00,
           -2.4291e+00, -2.4291e+00],
          [-2.4291e+00, -2.4291e+00, -2.4291e+00,  ..., -2.4291e+00,
           -2.4291e+00, -2.4291e+00],
          [-2.4291e+00, -2.4291e+00, -2.4291e+00,  ..., -2.4291e+00,
           -2.4291e+00, -2.4291e+00],
          ...,
          [-7.8135e-01, -3.1611e-01,  2.0728e-01,  ...,  7.5006e-01,
            8.0822e-01, -2.4291e+00],
          [-1.3823e+00, -7.8135e-01, -4.5180e-01,  ...,  5.5621e-01,
            3.0421e-01, -2.4291e+00],
          [-1.3823e+00, -1.2272e+00, -1.0334e+00,  ...,  2.6544e-01,
           -2.5336e-02, -2.4291e+00]],

         [[-2.4183e+00, -2.4183e+00, -2.4183e+00,  ..., -2.4183e+00,
           -2.4183e+00, -2.4183e+00],
          [-2.4183e+00, -2.4183e+00, -2.4183e+00,  ..., -2.4183e+00,
           -2.4183e+00, -2.4183e+00],
          [-2.4183e+00, -2.4183e+00, -2.4183e+00,  ..., -2.4183e+00,
           -2.418

...

          [-0.6411, -0.5825, -0.5435,  ...,  0.1198,  0.1979,  0.2759]]]])
tensor([[[[-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          ...,
          [-2.3903, -2.2740, -2.0220,  ..., -2.4291, -2.4291, -2.4291],
          [-2.2934, -2.3321, -2.2740,  ..., -2.4291, -2.4291, -2.4291],
          [-2.0801, -2.3515, -2.2740,  ..., -2.4291, -2.4291, -2.4291]],

         [[-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          ...,
          [-2.3199, -2.2806, -1.9266,  ..., -2.4183, -2.4183, -2.4183],
          [-2.0839, -2.3789, -2.2216,  ..., -2.4183, -2.4183, -2.4183],
          [-1.6709, -2.3789, -2.2413,  ..., -2.4183, -2.4183, -2.4183]],

         [[-2.2214, -2.2214

           -1.1428e-01, -2.2214e+00]]]])
tensor([[[[-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          ...,
          [-2.4291, -2.4291, -2.4291,  ..., -0.2192, -0.3355, -0.0835],
          [-2.4291, -2.4291, -2.4291,  ..., -0.2773, -0.2192, -0.0447],
          [-2.4291, -2.4291, -2.4291,  ..., -0.0060, -0.0641, -0.0060]],

         [[-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          ...,
          [-2.4183, -2.4183, -2.4183,  ..., -0.3926, -0.2942, -0.2549],
          [-2.4183, -2.4183, -2.4183,  ..., -0.4712, -0.3139, -0.2942],
          [-2.4183, -2.4183, -2.4183,  ..., -0.2156, -0.2746, -0.3336]],

         [[-2.2214, -2.2214, -2.2214,  ..., -2.2214, -2.2214,

          [-0.1338, -0.1533, -0.1728,  ..., -2.2214, -2.2214, -2.2214]]]])
tensor([[[[-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          ...,
          [-2.4291, -2.4291, -2.4291,  ...,  0.9827,  0.9633,  0.9633],
          [-2.4291, -2.4291, -2.4291,  ...,  0.9827,  0.9633,  0.9633],
          [-2.4291, -2.4291, -2.4291,  ...,  0.9827,  0.9633,  0.9439]],

         [[-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          ...,
          [-2.4183, -2.4183, -2.4183,  ...,  1.3184,  1.2988,  1.2988],
          [-2.4183, -2.4183, -2.4183,  ...,  1.3184,  1.2988,  1.2988],
          [-2.4183, -2.4183, -2.4183,  ...,  1.3184,  1.2988,  1.2791]],

         [[-2.2214, -2.2214

            6.4662e-01,  6.4662e-01]]]])
tensor([[[[-2.4291, -2.4291, -2.4291,  ..., -2.0026, -1.7894, -1.0140],
          [-2.4291, -2.4291, -2.4291,  ..., -1.4598, -1.5567, -0.8589],
          [-2.4291, -2.4291, -2.4291,  ..., -1.3241, -1.4211, -0.9170],
          ...,
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291]],

         [[-2.4183, -2.4183, -2.4183,  ..., -1.5726, -1.4546, -0.9826],
          [-2.4183, -2.4183, -2.4183,  ..., -1.1596, -1.3366, -0.8646],
          [-2.4183, -2.4183, -2.4183,  ..., -1.1989, -1.3562, -0.9629],
          ...,
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183]],

         [[-2.2214, -2.2214, -2.2214,  ..., -1.1483, -1.0898,

            8.0821e-02,  2.1739e-01]]]])


In [17]:
tqdm(train_loader)

  0%|<bar/>| 0/391 [00:00<?, ?it/s]

In [12]:
cut(inputs)

RuntimeError: The expanded size of the tensor (32) must match the existing size (3) at non-singleton dimension 2.  Target sizes: [80, 3, 32, 32].  Tensor sizes: [3, 32]

In [19]:
inputs

tensor([[[[-2.4291e+00, -2.4291e+00, -2.4291e+00,  ..., -2.4291e+00,
           -2.4291e+00, -2.4291e+00],
          [-2.4291e+00, -2.4291e+00, -2.4291e+00,  ..., -2.4291e+00,
           -2.4291e+00, -2.4291e+00],
          [-2.4291e+00, -2.4291e+00, -2.4291e+00,  ..., -2.4291e+00,
           -2.4291e+00, -2.4291e+00],
          ...,
          [-7.0381e-01, -5.4873e-01, -4.5180e-01,  ..., -1.3823e+00,
           -1.3435e+00, -1.3435e+00],
          [-6.2627e-01, -5.4873e-01, -5.6811e-01,  ..., -1.4792e+00,
           -1.4792e+00, -1.4792e+00],
          [-6.4565e-01, -7.0381e-01, -7.2319e-01,  ..., -1.3629e+00,
           -1.4211e+00, -1.4598e+00]],

         [[-2.4183e+00, -2.4183e+00, -2.4183e+00,  ..., -2.4183e+00,
           -2.4183e+00, -2.4183e+00],
          [-2.4183e+00, -2.4183e+00, -2.4183e+00,  ..., -2.4183e+00,
           -2.4183e+00, -2.4183e+00],
          [-2.4183e+00, -2.4183e+00, -2.4183e+00,  ..., -2.4183e+00,
           -2.4183e+00, -2.4183e+00],
          ...,
     

In [24]:
for i in range(inputs.size(0)):
    if random.random()<0.5:
        inputs[i] = cut(inputs[i])


torch.Size([80, 3, 32, 32])